<h1>Preparing</h1>

In [1]:
# Necessary libraries.
import os, subprocess, uuid
import regex as re
import datetime
from time import sleep

import numpy as np, pandas as pd

from tqdm import tqdm

import json, csv
from typing import List, Tuple, Set
from pathlib import Path

from py2neo import Graph
from neo4j.exceptions import ServiceUnavailable, TransientError

import shutil
import reverse_geocoder as rg

In [44]:
# Const.
IN_CATEGORY = "IN_CATEGORY"
IN_CITY = "IN_CITY"
IN_AREA = "IN_STATE"
IN_COUNTRY = "IN_COUNTRY"
HAS = "HAS"
FRIENDS = "FRIENDS"
REVIEWS = "REVIEWS_FOR"
WROTE = "WROTE"
BUSINESS_NODE = "Restaurant"
BUSINESS_ATTRS_NODE = "B_Attribute"
USER_NODE = "User"
REVIEW_NODE = "Review"
CATEGORY_NODE = "Category"
MENU_NODE = "Menu"
PHOTO_NODE = "Photo"
CITY_NODE = "City"
AREA_NODE = "State"
COUNTRY_NODE = "Country"

CATEGORY_FILTER = ["Restaurants", "Food", "Fast Food", "Sandwiches", "Breakfast & Brunch", 
                   "Coffee & Tea", "American", "Burgers", "Mexican", "Desserts", 
                   "Specialty Food", "Pizza", "Salad", "Italian", "Chicken Wings",
                  "Tacos", "Seafood", "Vegetarian", "Ethnic Food", "Chinese", 
                   "Barbeque", "Mediterranean", "Japanese", "Korean", "Indian"]

In [2]:
# Important locations.
data_folder = r"../../Dataset/yelp"
business_json_file = data_folder + "/json/yelp_academic_dataset_business.json"
review_json_file = data_folder + "/json/yelp_academic_dataset_review.json"
user_json_file = data_folder + "/json/yelp_academic_dataset_user.json"
checkin_json_file = data_folder + "/json/yelp_academic_dataset_checkin.json"
business_attrs_json_file = data_folder + "/json/bussiness_attrs.json"
photos_json_file = data_folder + "/photos/photos.json"
fixed_business_json_file = data_folder + "/json/fixed_yelp_academic_dataset_business.json"
fixed_review_json_file = data_folder + "/json/fixed_yelp_academic_dataset_review.json"
fixed_user_json_file = data_folder + "/json/fixed_yelp_academic_dataset_user.json"
fixed_checkin_json_file = data_folder + "/json/fixed_yelp_academic_dataset_checkin.json"
fixed_photos_json_file = data_folder + "/json/fixed_photos.json"
fixed_business_attrs_json_file = data_folder +  "/json/fixed_bussiness_attrs.json"
list_raw_files = [business_json_file, review_json_file, user_json_file, checkin_json_file, 
                  business_attrs_json_file, photos_json_file]
list_fixed_data = [fixed_business_json_file, fixed_review_json_file, fixed_user_json_file, 
                   fixed_checkin_json_file, fixed_photos_json_file, fixed_business_attrs_json_file]

In [44]:
# Csv files for Import Tool.
# business_nodes_csv_file = data_folder + "/neo4j/csv/business_nodes.csv"
# category_nodes_csv_file = data_folder + "/neo4j/csv/category_nodes.csv"
# city_nodes_csv_file = data_folder + "/neo4j/csv/city_nodes.csv"
# area_nodes_csv_file = data_folder + "/neo4j/csv/area_nodes.csv"
# country_nodes_csv_file = data_folder + "/neo4j/csv/country_nodes.csv"
# user_nodes_csv_file = data_folder + "/neo4j/csv/user_nodes.csv"
# review_nodes_csv_file = data_folder + "/neo4j/csv/review_nodes.csv"
# relationship_csv_file = data_folder + "/neo4j/csv/relationships.csv"
# nodes_files = [business_nodes_csv_file, category_nodes_csv_file, city_nodes_csv_file, 
#                area_nodes_csv_file, country_nodes_csv_file, user_nodes_csv_file, review_nodes_csv_file]

# business_nodes_csv_file = data_folder + "/neo4j/csv/restaurant_nodes.csv"
# business_attrs_nodes_csv_file = data_folder + "/neo4j/csv/business_attr_node.csv"
# category_nodes_csv_file = data_folder + "/neo4j/csv/category_nodes.csv"
# photo_nodes_csv_file = data_folder + "/neo4j/csv/photo_nodes.csv"
# city_nodes_csv_file = data_folder + "/neo4j/csv/city_nodes.csv"
# area_nodes_csv_file = data_folder + "/neo4j/csv/state_nodes.csv"
# country_nodes_csv_file = data_folder + "/neo4j/csv/country_nodes.csv"
# user_nodes_csv_file = data_folder + "/neo4j/csv/user_nodes.csv"
# review_nodes_csv_file = data_folder + "/neo4j/csv/review_nodes.csv"
# relationship_csv_file = data_folder + "/neo4j/csv/relationships.csv"
# relationship_business_attrs_csv_file = data_folder + "/neo4j/csv/relationships_business_attrs.csv"
# nodes_files = [business_nodes_csv_file, business_attrs_nodes_csv_file, category_nodes_csv_file, 
#                photo_nodes_csv_file, city_nodes_csv_file, area_nodes_csv_file, country_nodes_csv_file, 
#                user_nodes_csv_file,  review_nodes_csv_file]

restaurant_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurants.csv"
category_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/category.csv"
city_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/city.csv"
menu_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/menu.csv"
state_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/state.csv"
country_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/country.csv"
user_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/user.csv"
source_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/source.csv"
review_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/review.csv"
restaurant_attr_node_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurant_attr.csv"
relationship_csv_file = "/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/relation.csv"
nodes_files = [restaurant_node_csv_file, category_node_csv_file, city_node_csv_file, menu_node_csv_file,
               state_node_csv_file, country_node_csv_file, user_node_csv_file, source_node_csv_file,
               review_node_csv_file, restaurant_attr_node_csv_file]

In [46]:
# Connection to Neo4j Db.
graph_name = "neo4j" # Default for 4.0: neo4j; default for 3.5: graph.db
SERVER_ADDRESS = "bolt://localhost:7687"
SERVER_AUTH = ("neo4j", "erclab")
graph = Graph(SERVER_ADDRESS, auth=SERVER_AUTH)

In [47]:
neo4j_home = graph.service.config["dbms.directories.neo4j_home"]

In [48]:
print(neo4j_home)

/home/muzamil/.config/Neo4j Desktop/Application/neo4jDatabases/database-78a2aebd-c68b-431c-bc51-61237ddf706a/installation-4.1.0


In [49]:
# Check neo4j connection
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')

ok


In [7]:
def delete_files(files: List[str]):
    for one_file in files:
        one_path = Path(one_file)
        if one_path.is_file():
            one_path.unlink()

In [ ]:
# if not all(list(map(lambda x: Path(x).is_file(), list_raw_files))):
#     raise Exception(f'Not all Yelp raw files are available. Need: {list_raw_files}')

<h1>Preprocessing Data</h1>

<b>Data Problems:</b>

    1. Ids are unique only in the specific domain. E.g. business_id in Business oiAlXZPIFm2nBCt0DHLu_Q is identical to that user_id in User. When using Neo4j's Import Tool, the relationship csv defines "from-node" and "to-node", identified by unique ids. Thus, the entities' ids, regardless of Business or User or Review must be unique.
    2. Users have friends, but not all the friends exist. E.g. Unknown friend: oeMvJh94PiGQnx_6GlndPQ for User ntlvfPzc8eglqvk92iDIAw. Those unknown friends must be removed.
    3. Some Business contain duplicate categories, e.g. Business with id HEGy1__jKyMhkhXRW3O1ZQ has duplicated Gas Stations. These must be deduplicated.
    4. In Users, friends is supposed to be an array with string elements. But in reality, it is a string in which friend_ids are concatenated with commas. Same problem with categories field in Business.

In [22]:
# Fix data problems.
def remove_unknown_friends(raw_path: str, output_path: str) -> None:
    user_ids = set()
    with open(raw_path, "r", encoding="utf-8") as rf:
        for line in rf:
            if len(line.strip()) > 0:
                json_node = json.loads(line)
                user_ids.add(json_node["user_id"])
            
    with open(raw_path, "r", encoding="utf-8") as rf, open(output_path, mode="w", encoding="utf-8") as of:
        for line in rf:
            if len(line.strip()) > 0:
                json_node = json.loads(line)
                friends_str = json_node["friends"]
                if friends_str and len(friends_str.strip()) > 0:
                    friends_arr = re.split(r"\s*,\s*", friends_str.strip())
                    friends_exist_arr = list(filter(lambda x: x in user_ids, friends_arr))
                    json_node["friends"] = ', '.join(friends_exist_arr)
                    of.write(f'{json.dumps(json_node)}\n')

                    
def make_ids_unique(input_path: str, output_path: str, func_to_modify) -> None:
    temp_output_file = str(uuid.uuid4())
    line_count = len(open(input_path).readlines())
    with open(input_path, mode="r", encoding="utf-8") as in_f, open(temp_output_file, mode="w", encoding="utf-8") as ou_f:
        for line in tqdm(in_f, total=line_count):
            if len(line.strip())>0:
                json_node = json.loads(line)
                json_node = func_to_modify(json_node)
                if json_node is not None :
                    ou_f.write(f'{json.dumps(json_node)}\n')
    # Rename the file.
    os.replace(temp_output_file, output_path)

In [6]:
def get_food_businesses():
    business_ids = []
    line_count = len(open(fixed_business_json_file).readlines())
    with open(fixed_business_json_file) as ref:
        for line in tqdm(ref, total=line_count):
            blob = json.loads(line)
            business_ids += [blob["business_id"]]
        print(str(len(business_ids))+" "+business_ids[0])
    return business_ids

business_ids = get_food_businesses()

100%|██████████| 80455/80455 [00:00<00:00, 89437.67it/s]

80455 b-pQeaRpvuhoEqudo3uymHIQ


In [ ]:
def get_foodie_reviewer():
    user_ids = []
    line_count = len(open(fixed_review_json_file).readlines())
    with open(fixed_review_json_file) as ref:
        for line in tqdm(ref, total=line_count):
            blob = json.loads(line)
            user_ids += [blob["user_id"]]
        print(str(len(user_ids))+" "+user_ids[0])
    return user_ids

user_ids = get_foodie_reviewer()

In [ ]:
df = pd.DataFrame(user_ids)

In [ ]:
def fix_user(json_node):
        json_node["user_id"] = "u-" + json_node["user_id"]
        friends_str = json_node["friends"]
        if friends_str and len(friends_str.strip()) > 0:
            friends_arr = re.split(r"\s*,\s*", friends_str.strip())
            friends_arr = list(map(lambda x: "u-" + x, friends_arr))
            json_node["friends"] = ', '.join(friends_arr)
        return json_node
#         if df[0].eq(json_node["user_id"]).any():
#             return json_node
#         else :
#             return None

In [18]:
make_ids_unique(fixed_user_json_file, fixed_user_json_file, fix_user)

100%|██████████| 1968703/1968703 [00:58<00:00, 33936.85it/s]


In [44]:
attrs_keys = ["RestaurantsDelivery", "RestaurantsGoodForGroups", "RestaurantsTakeOut", "WiFi", 
              "BusinessAcceptsCreditCards", "BusinessParking", "BikeParking", "GoodForKids"]
def fix_business_attrs(input_path: str, output_path: str):
    temp_output_file = str(uuid.uuid4())
    with open(input_path) as json_file, open(temp_output_file, mode="w", encoding="utf-8") as ou_f:
        data = json.load(json_file)
        for business in data:
            if "attributes" in business:
                for key, value in list(business["attributes"].items()):
                    if key not in attrs_keys:
                        del business["attributes"][key]
                        continue
                        
                    if "RestaurantsDelivery" == key:
                        if value != "True":
                            del business["attributes"][key]
                    if "RestaurantsGoodForGroups" == key:
                        if value != "True":
                            del business["attributes"][key]
                    if "RestaurantsTakeOut" == key:
                        if value != "True":
                            del business["attributes"][key]
                    if "WiFi" == key:
                        if value != "free":
                            del business["attributes"][key]
                    if "BusinessAcceptsCreditCards" == key:
                        if value != "True":
                            del business["attributes"][key]
                    if "BusinessParking" == key:
                        if value == "None":
                            del business["attributes"][key]
                        else :
                            business["attributes"][key] = "True"
                    if "BikeParking" == key:
                        if value != "True":
                            del business["attributes"][key]
                    if "GoodForKids" == key:
                        if value != "True":
                            del business["attributes"][key]

                ou_f.write(f'{json.dumps(business)}\n')
    # Rename the file.
    os.replace(temp_output_file, output_path)
    
fix_business_attrs(business_attrs_json_file, fixed_business_attrs_json_file)

In [24]:
def get_users_with_review(input_path: str, output_path: str) -> None:
    temp_output_file = str(uuid.uuid4())
    line_count = len(open(input_path).readlines())
    with open(input_path, mode="r", encoding="utf-8") as in_f, open(temp_output_file, mode="w", encoding="utf-8") as ou_f:
        for line in tqdm(in_f, total=line_count):
            if len(line.strip())>0:
                json_node = json.loads(line)
                json_df = pd.DataFrame([json_node])
                if df[0].eq(json_df["user_id"]).any():
                    ou_f.write(f'{json.dumps(json_node)}\n')
    # Rename the file.
    os.replace(temp_output_file, output_path)

In [ ]:
get_users_with_review(fixed_user_json_file, fixed_user_json_file)

In [11]:
# Check if the fixed files are already available.
if not all(list(map(lambda x: Path(x).is_file(), list_fixed_data))):
    # Remove everything first.
    delete_files(list_fixed_data)
    # Re-create the fixed files.
    def fix_user(json_node):
        json_node["user_id"] = "u-" + json_node["user_id"]
        friends_str = json_node["friends"]
        if friends_str and len(friends_str.strip()) > 0:
            friends_arr = re.split(r"\s*,\s*", friends_str.strip())
            friends_arr = list(map(lambda x: "u-" + x, friends_arr))
            json_node["friends"] = ', '.join(friends_arr)
        return json_node
        
    def fix_business(json_node):
        json_node["business_id"] = "b-" + json_node["business_id"]
        if json_node["categories"] and len(json_node["categories"].strip())>0:
            categories_arr = re.split("\s*,\s*", json_node["categories"].strip())
            categories_set = set(categories_arr)
            category_food_filter = list(filter(lambda x: x in CATEGORY_FILTER, categories_set))
            if len(category_food_filter) > 0 :
                json_node["categories"] = ', '.join(categories_set)
                return json_node
            else :
                return None
            
    def fix_review(json_node):
        json_node["review_id"] = "r-" + json_node["review_id"]
        json_node["user_id"] = "u-" + json_node["user_id"]
        json_node["business_id"] = "b-" + json_node["business_id"]
        if any(json_node["business_id"] in s for s in business_ids):
            return json_node
        else :
            return None
        
    def fix_checkin(json_node):
        json_node["business_id"] = "b-" + json_node["business_id"]
        if any(json_node["business_id"] in s for s in business_ids):
            return json_node
        else :
            return None
        
    def fix_photos(json_node):
        json_node["photo_id"] = "p-" + json_node["photo_id"]
        json_node["business_id"] = "b-" + json_node["business_id"]
        if any(json_node["business_id"] in s for s in business_ids):
            return json_node
        else :
            return None

    # Start fixing data problems.
#     remove_unknown_friends(user_json_file, fixed_user_json_file)
#     make_ids_unique(fixed_user_json_file, fixed_user_json_file, fix_user)
#     make_ids_unique(business_json_file, fixed_business_json_file, fix_business)
#     make_ids_unique(review_json_file, fixed_review_json_file, fix_review)
#     make_ids_unique(checkin_json_file, fixed_checkin_json_file, fix_checkin)
#     make_ids_unique(photos_json_file, fixed_photos_json_file, fix_photos)
    make_ids_unique(photos_json_file, fixed_photos_json_file, fix_photos)

In [12]:
def fix_photos(json_node):
        json_node["photo_id"] = "p-" + json_node["photo_id"]
        json_node["business_id"] = "b-" + json_node["business_id"]
        if any(json_node["business_id"] in s for s in business_ids):
            return json_node
        else :
            return None

make_ids_unique(photos_json_file, fixed_photos_json_file, fix_photos)

100%|██████████| 200000/200000 [12:36<00:00, 264.43it/s]


<h1>Importing Data</h1>

<h3>Generating inputs for Import Tool</h3>

In [45]:
# importing business attrs relation for neo4j

# Relationship writer.
relationship_business_attrs_csv = open(relationship_business_attrs_csv_file, mode="w", 
                                       encoding="utf-8", newline="\n")
relationship_business_attrs_fieldnames = [":START_ID", ":END_ID", ":TYPE"]
relationship_business_attrs_writer = csv.DictWriter(relationship_business_attrs_csv, 
                                                    fieldnames=relationship_business_attrs_fieldnames, 
                                     delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
relationship_business_attrs_writer.writeheader()

def write_business_attr_nodes_to_file():
    with open(fixed_business_attrs_json_file, "r", encoding="utf-8") as bajf:

        # File too large, read line by line.
        for line in bajf:
            line = line.strip()
            if len(line)>0:
                json_node = json.loads(line)

                for key, value in list(json_node["attributes"].items()):
                    relationship_business_attrs_writer.writerow({k:v for k, v in 
                                                                 zip(relationship_business_attrs_fieldnames, 
                                                                          [json_node["business_id"], 
                                                                           key, HAS])})
                
    
write_business_attr_nodes_to_file()

In [47]:
if not all(list(map(lambda x: Path(x).is_file(), nodes_files))) or not Path(relationship_csv_file).is_file():
    # Delete everything.
    delete_files(nodes_files)
    delete_files([relationship_csv_file])
    # Using sets to ensures uniqueness.
    business_lat_lon = {}
    area_nodes: Set[Tuple[str, str]] = set()
    city_nodes: Set[Tuple[str, str]]= set()
    country_nodes: Set[str] = set()
    categories_nodes: Set[str] = set()
    
    in_area_relationships: Set[Tuple[str, str, str]] = set()
    in_country_relationships: Set[Tuple[str, str, str]] = set()
    # Relationship writer.
    relationship_csv = open(relationship_csv_file, mode="w", encoding="utf-8", newline="\n")
    relationship_fieldnames = [":START_ID", ":END_ID", ":TYPE"]
    relationship_writer = csv.DictWriter(relationship_csv, fieldnames=relationship_fieldnames, 
                                         delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    relationship_writer.writeheader()
    
    # Func to write Business Nodes to file.
    def write_business_nodes_to_file():
        with open(fixed_business_json_file, "r", encoding="utf-8") as bjf, open(
            business_nodes_csv_file, mode="w", encoding="utf-8", newline="\n") as business_nodes_csv:
            fieldnames = ["business_id:ID", "name", "address", "postal_code", "latitude", "longitude", 
                          "stars", "review_count", ":LABEL"]
            writer = csv.DictWriter(business_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            # File too large, read line by line.
            for line in bjf:
                line = line.strip()
                if len(line)>0:
                    json_node = json.loads(line)

                    writer.writerow({k:v for k, v in zip(fieldnames, [json_node["business_id"], 
                                                                      json_node["name"], json_node["address"],
                                                                      json_node["postal_code"], 
                                                                      json_node["latitude"], 
                                                                      json_node["longitude"],
                                                                      json_node["stars"], 
                                                                      json_node["review_count"],BUSINESS_NODE])})
                    business_lat_lon[json_node["business_id"]] = (json_node["latitude"], json_node["longitude"])
                            
                    if json_node["categories"] and len(json_node["categories"].strip())>0: 
                        # can be None (e.g. for Business Id: 2W1tLg8ybRUEKMPoAPHTsQ)
                        cur_categories = list(filter(lambda x: len(x)>0, 
                                                     map(lambda x: x.strip(), 
                                                         re.split("\s*,\s*", json_node["categories"].strip()))))
                        categories_nodes.update(cur_categories)
                        for category in cur_categories:
                            relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                                              [json_node["business_id"], 
                                                                               category, IN_CATEGORY])})
    
    # Func to write Category Nodes to file.
    def write_category_nodes_to_file():
        with open(category_nodes_csv_file, mode="w", encoding="utf-8", newline="\n") as category_nodes_csv:
            fieldnames = ["category_id:ID", ":LABEL"]
            writer = csv.DictWriter(category_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            for category_id in categories_nodes:
                writer.writerow({k:v for k, v in zip(fieldnames, [category_id, CATEGORY_NODE])})
    
    def write_business_attr_nodes_to_file():
        with open(fixed_business_attrs_json_file, "r", encoding="utf-8") as bajf:

            # File too large, read line by line.
            for line in bajf:
                line = line.strip()
                if len(line)>0:
                    json_node = json.loads(line)
                    for key, value in list(json_node["attributes"].items()):
                        relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                                              [json_node["business_id"], 
                                                                               key, HAS])})
    # Func to make City, Area, Country Nodes.
    def make_city_area_country_nodes():
        lat_lons = list(business_lat_lon.values())
        city_state_countries = rg.search(lat_lons)
        # Process City, Area, Country Nodes; save IN_CITY relationship to file.
        for (business_id, city_state_country) in list(zip(list(business_lat_lon.keys()), city_state_countries)):
            city, state, country = city_state_country["name"], city_state_country["admin1"], city_state_country["cc"]
            unique_state = f'{state}-{country}'
            unique_city = f'{city}-{state}-{country}'
            country_nodes.add(country)
            area_nodes.add((unique_state, state))
            city_nodes.add((unique_city, city))
            # Create corresponding relationships.
            relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                              [business_id, unique_city, IN_CITY])})
            in_area_relationships.add((unique_city, unique_state, IN_AREA))
            in_country_relationships.add((unique_state, country, IN_COUNTRY))
    
    # Func to write City, Area, Country Nodes to file.
    def write_city_area_country_nodes_to_file():
        with open(city_nodes_csv_file, mode="w", encoding="utf-8", newline="\n") as city_nodes_csv:
            fieldnames = ["city_id:ID", "name", ":LABEL"]
            writer = csv.DictWriter(city_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            for (city_id, city_name) in city_nodes:
                writer.writerow({k:v for k, v in zip(fieldnames, [city_id, city_name, CITY_NODE])})

        with open(area_nodes_csv_file, mode="w", encoding="utf-8", newline="\n") as area_nodes_csv:
            fieldnames = ["area_id:ID", "name", ":LABEL"]
            writer = csv.DictWriter(area_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            for (area_id, area_name) in area_nodes:
                writer.writerow({k:v for k, v in zip(fieldnames, [area_id, area_name, AREA_NODE])})

        with open(country_nodes_csv_file, mode="w", encoding="utf-8", newline="\n") as country_nodes_csv:
            fieldnames = ["country_id:ID", ":LABEL"]
            writer = csv.DictWriter(country_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            for country_id in country_nodes:
                writer.writerow({k:v for k, v in zip(fieldnames, [country_id, COUNTRY_NODE])})
                
        for (u1, u2, rel_type) in in_area_relationships:
            relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, [u1, u2, rel_type])})

        for (u1, u2, rel_type) in in_country_relationships:
            relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, [u1, u2, rel_type])})
    
    # Func to write Review nodes to file.
    def write_photo_nodes_to_file():
        with open(fixed_photos_json_file, "r", encoding="utf-8") as rjf, open(photo_nodes_csv_file, 
                                                                              mode="w", encoding="utf-8", 
                                                                              newline="\n") as checkin_nodes_csv:
            fieldnames = ["photo_id:ID", "caption", "label", ":LABEL"]
            writer = csv.DictWriter(checkin_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            # File too large, read line by line.
            for line in rjf:
                line = line.strip()
                if len(line)>0:
                    json_node = json.loads(line)
                    writer.writerow({k:v for k, v in zip(fieldnames, [json_node["photo_id"], json_node["caption"],
                                                                      json_node["label"], PHOTO_NODE])})
                    relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                                      [json_node["business_id"], 
                                                                       json_node["photo_id"], HAS])})
                    
    # Func to write User nodes to file.
    def write_user_nodes_to_file():
        friend_relationships: Set[Tuple[str, str, str]] = set()
        with open(fixed_user_json_file, "r", encoding="utf-8") as ujf, open(user_nodes_csv_file, 
                                                                            mode="w", encoding="utf-8", 
                                                                            newline="\n") as user_nodes_csv:
            fieldnames = ["user_id:ID", "name", "review_count", "yelping_since", "average_star", ":LABEL"]
            writer = csv.DictWriter(user_nodes_csv, fieldnames=fieldnames, delimiter=",", quotechar='"', 
                                    quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            # File too large, read line by line.
            for line in ujf:
                line = line.strip()
                if len(line)>0:
                    json_node = json.loads(line)
                    writer.writerow({k:v for k, v in zip(fieldnames, [json_node["user_id"], 
                                                                      json_node["name"], 
                                                                      json_node["review_count"],
                                                                      json_node["yelping_since"], 
                                                                      json_node["average_stars"], 
                                                                      USER_NODE])})
                    if json_node["friends"] and len(json_node["friends"].strip()) > 0:
                        friends_arr = re.split("\s*,\s*", json_node["friends"].strip())
                        for friend_id in friends_arr:
                            # Prevent duplicate friend relationship later!
                            f1 = min(json_node["user_id"], friend_id)
                            f2 = max(json_node["user_id"], friend_id)
                            friend_relationships.add((f1, f2, FRIENDS))
        
        for (f1, f2, rel_type) in friend_relationships:
            relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, [f1, f2, rel_type])})
    
    # Func to write Review nodes to file.
    def write_review_nodes_to_file():
        with open(fixed_review_json_file, "r", encoding="utf-8") as rjf, open(review_nodes_csv_file, 
                                                                              mode="w", encoding="utf-8", 
                                                                              newline="\n") as review_nodes_csv:
            fieldnames = ["review_id:ID", "stars", "date", "text", ":LABEL"]
            writer = csv.DictWriter(review_nodes_csv, fieldnames=fieldnames, delimiter=",", 
                                    quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            # File too large, read line by line.
            for line in rjf:
                line = line.strip()
                if len(line)>0:
                    json_node = json.loads(line)
                    writer.writerow({k:v for k, v in zip(fieldnames, [json_node["review_id"], 
                                                                      json_node["stars"], json_node["date"], 
                                                                      json_node["text"], REVIEW_NODE])})
                    relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                                      [json_node["user_id"], 
                                                                       json_node["review_id"], WROTE])})
                    relationship_writer.writerow({k:v for k, v in zip(relationship_fieldnames, 
                                                                      [json_node["review_id"], 
                                                                       json_node["business_id"], REVIEWS])})
    
    # Start re-creating CSV files for Import Tool.
    write_business_nodes_to_file()
    write_category_nodes_to_file()
    write_business_attr_nodes_to_file()
    del categories_nodes
    make_city_area_country_nodes()
    write_city_area_country_nodes_to_file()
    write_photo_nodes_to_file()
    del business_lat_lon, city_nodes, area_nodes, country_nodes, in_area_relationships, in_country_relationships
    write_user_nodes_to_file()
    write_review_nodes_to_file()
    
    relationship_csv.close()

<h3>Checking Data Integrity</h3>

In [54]:
num_of_nodes = 0

# Func to if nodes are unique and relationships are also unique.
def check_nodes_relationships_csv_files_integrity():
    global num_of_nodes
    # Nodes files.
    for one_node_file in nodes_files:
        print("one_node_file: "+one_node_file)
        temp_df = pd.read_csv(one_node_file, header = "infer", sep = ",", encoding = "utf-8")
        if len(temp_df.iloc[:, 0]) != len(np.unique(temp_df.iloc[:, 0].values)):
            raise Exception(f'Nodes in [{one_node_file}]: not unique')
        num_of_nodes += len(temp_df.iloc[:, 0])
    # Relationship file.
    temp_df = pd.read_csv(relationship_csv_file, header = "infer", sep = ",", encoding = "utf-8")
    rels = temp_df.iloc[:, 0] + temp_df.iloc[:, 1]
    if len(rels) != len(np.unique(rels.values)):
        raise Exception(f'Relationships in [{relationship_csv_file}]: not unique')

In [55]:
check_nodes_relationships_csv_files_integrity()

one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurants.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/category.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/city.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/menu.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/state.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/country.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/user.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/source.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/review.csv
one_node_file: /home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurant_attr.csv


<h3>Importing to Neo4j</h3>

<b>Reseting the database and importing data</b>

The quickest/cleanest/safest way is:

    1. Stop the Neo4j's Database Service.
    2. Remove the neo4j's Database's Data folder.
    3. Execute the Import Tool.
    4. Start the Neo4j's Database Service.

<b>Important Note:</b> Users must have privileges to start/stop Windows services. If needing elevating, a Windows popup will appear, asking for the permission.

In [56]:
# Func to control Neo4j's Database Service.
def command_neo4j_database_service(cmd: str):
    neo4j_cmd = neo4j_home + r"/bin/neo4j"
    if cmd in ["stop", "start"]:
        cmd_res = subprocess.run([neo4j_cmd, cmd], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        cmd_stdout = str(cmd_res.stdout, "utf-8")
        print(cmd_stdout)
#         if cmd == "stop" and "stopped" not in cmd_stdout:
#             raise Exception(f"Can't stop Neo4j's Database Service [{cmd_res.stderr}]")
#         elif cmd == "start" and "started" not in cmd_stdout:
#             raise Exception(f"Can't start Neo4j's Database Service [{cmd_res.stderr}]")
    else:
        raise Exception(f'Unknown command for Neo4j\'s Database Service: [{cmd}]')

In [57]:
# Func to remove the Database's Data folder.
def reset_neo4j_database():
    neo4j_graph_folder = rf"{neo4j_home}/data/databases/{graph_name}"
    neo4j_trans_folder = rf"{neo4j_home}/data/transactions/{graph_name}"
    print(neo4j_graph_folder)
    print(neo4j_trans_folder)
    if not Path(neo4j_graph_folder).is_dir():
        raise Exception("Can't find Neo4j's Database's Data folder")
    shutil.rmtree(neo4j_graph_folder)
    shutil.rmtree(neo4j_trans_folder)

In [58]:
# Func to import data into Neo4j's Database.
def import_data():
    import_tool_cmd = neo4j_home + r"/bin/neo4j-admin"
    arguments = ["import", "--multiline-fields=true"] + list(map(lambda x: f'--nodes={x}', nodes_files)) + [f"--relationships={relationship_csv_file}"]
    print(arguments)
    # Execute the Import Tool (This tool will re-create a fresh "neo4j" folder if needed).
    cmd_res = subprocess.run([import_tool_cmd] + arguments, cwd=os.getcwd(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(str(cmd_res.stdout, "utf-8"))
    if "IMPORT DONE" not in str(cmd_res.stdout, "utf-8"):
        raise Exception(f"Can't execute Import Tool successfully [{cmd_res.stderr}]")

In [61]:
# Check if Neo4j's Database Service is already containing the desired data.
if not(len(graph.nodes) == num_of_nodes and num_of_nodes > 0):
    # Start resetting the database, then importing the data (nodes & relationships).
#     command_neo4j_database_service("stop")
#     reset_neo4j_database()
    import_data()
#     command_neo4j_database_service("start")

['import', '--multiline-fields=true', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurants.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/category.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/city.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/menu.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/state.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/country.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/user.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/source.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/review.csv', '--nodes=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/restaurant_attr.csv', '--relationships=/home/muzamil/Projects/Python/ML/NLP/KG/Restaurant_KG/dataset/relation.csv']
Neo4j version: 4.1

In [62]:
yelp_graph_ready = False

# Func to check if the importing process was done successfully.
def check_if_importing_is_successful():
    global graph
    global yelp_graph_ready
    num_tries = 30
    for one_try in range(num_tries):
        try:
            graph = Graph(SERVER_ADDRESS, auth=SERVER_AUTH)
            cur_num_nodes = len(graph.nodes)
            if cur_num_nodes == 0:
                raise Exception(f"There is no node in the Database")
            if cur_num_nodes != num_of_nodes:
                raise Exception(f'Expected: [{num_of_nodes}] nodes, but found: [{cur_num_nodes}]')
            yelp_graph_ready = True
        except (ConnectionRefusedError, ServiceUnavailable, TransientError, ConnectionAbortedError) as e:
            print(f"{e}. Try again...")
            sleep(1)

In [64]:
check_if_importing_is_successful()
if not yelp_graph_ready:
    raise Exception("Tried waiting for Neo4j Database Service, but still not available")

Exception: There is no node in the Database